In [2]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time
import re
import dateutil.parser
from functions import (get_sector, changes_from_press, get_content_header, 
                       get_df, normalise_to_index, save_data)


In [3]:
year = datetime.today().year

driver = webdriver.Firefox()
driver.get('https://investors.ball.com/news-and-presentations/news-releases/default.aspx')

data = []

ticker = 'ball'
sector = get_sector(ticker)
index_ticker = '^GSPC'
 

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)

year = datetime.today().year

while True:
    if year == 2013:
        break
    
    select = Select(driver.find_element(By.ID, "newsYear"))
    try:
        select.select_by_value(str(year))
    except:
        break

    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html) 

    articles = soup.find_all('div',attrs={'class':'module_item'})
    
    for article in articles:
        date = article.find('div', attrs={'class':'module_date-time'}).text.lstrip().rstrip() 
        title = article.find('div', attrs={'class':'module_headline'}).text.rstrip() 
        url = article.find('a').get('href')
        url = f'https://investors.ball.com{url}'
        
        title = re.sub("\s\s+", " ", title)

        date = dateutil.parser.parse(date).date()
        
        content = get_content_header(url, 'meta', {'name':'twitter:description'})
        pct_change = changes_from_press(stock_hist, date, 1)
        
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([str(date), title, url, content, pct_change, index_price, norm_price])
    year = year - 1
    

driver.close()

save_data(data, ticker, sector)

NoSuchElementException: Message: Unable to locate element: [id="newsYear"]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:395:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:134:16


In [ ]:
soup